In [2]:
import pandas as pd
import numpy as np
import os, os.path
import sqlite3
import warnings
warnings.filterwarnings('ignore')

## Projekt nr 4 - ramki danych w Python'ie

### Autor: Zuzanna Piróg

Niniejsza praca stanowi rozwiązanie czwartego projektu z PDU. Zadanie polegało na stworzeniu ramek danych odpowiadającym poleceniom SQL'owym przy pomocy pakietu Pandas w Pythonie. Poprawność będziemy sprawdzać przy pomocy funkcji .equals()

In [3]:
#Wczytujemy wszystkie ramki
Badges = pd.read_csv("Badges.csv.gz",
                     compression= 'gzip')
Comments = pd.read_csv("Comments.csv.gz",
                     compression= 'gzip')
PostLinks = pd.read_csv("PostLinks.csv.gz",
                     compression= 'gzip')
Posts = pd.read_csv("Posts.csv.gz",
                     compression= 'gzip')
Tags = pd.read_csv("Tags.csv.gz",
                     compression= 'gzip')
Users = pd.read_csv("Users.csv.gz",
                     compression= 'gzip')
Votes = pd.read_csv("Votes.csv.gz",
                     compression= 'gzip')

Stworzmy sciezke dostepu do bazy danych

In [4]:
baza = os.path.join('baza_danych.db')
if os.path.isfile(baza):
    os.remove(baza)

conn = sqlite3.connect(baza)
Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

In [5]:
df_sql_1 = pd.read_sql_query("""SELECT Count, TagName
               FROM Tags
               WHERE Count > 1000
               ORDER BY Count DESC""", conn)
df_sql_2 = pd.read_sql_query("""SELECT Location, COUNT(*) AS Count
                FROM (
                SELECT Posts.OwnerUserId, Users.Id, Users.Location
                FROM Users
                JOIN Posts ON Users.Id = Posts.OwnerUserId
                )
                WHERE Location NOT IN ('')
                GROUP BY Location
                ORDER BY Count DESC
                LIMIT 10""", conn)
df_sql_3 = pd.read_sql_query("""SELECT Year, SUM(Number) AS TotalNumber
                FROM (
                SELECT
                Name,
                COUNT(*) AS Number,
                STRFTIME('%Y', Badges.Date) AS Year
                FROM Badges
                WHERE Class = 1
                GROUP BY Name, Year
                )
                GROUP BY Year
                ORDER BY TotalNumber""", conn)
df_sql_4 = pd.read_sql_query("""SELECT
                Users.AccountId,
                Users.DisplayName,
                Users.Location,
                AVG(PostAuth.AnswersCount) as AverageAnswersCount
                FROM
                (
                SELECT
                AnsCount.AnswersCount,
                Posts.Id,
                Posts.OwnerUserId
                FROM (
                SELECT Posts.ParentId, COUNT(*) AS AnswersCount
                FROM Posts
                WHERE Posts.PostTypeId = 2
                GROUP BY Posts.ParentId
                ) AS AnsCount
                JOIN Posts ON Posts.Id = AnsCount.ParentId
                ) AS PostAuth
                JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
                GROUP BY OwnerUserId
                ORDER BY AverageAnswersCount DESC, AccountId ASC
                LIMIT 10""", conn)
df_sql_5 = pd.read_sql_query("""SELECT Posts.Title, Posts.Id,
                STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date,
                VotesByAge.Votes
                FROM Posts
                JOIN (
                SELECT
                PostId,
                MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
                MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
                SUM(Total) AS Votes
                FROM (
                SELECT
                PostId,
                CASE STRFTIME('%Y', CreationDate)
                WHEN '2021' THEN 'new'
                WHEN '2020' THEN 'new'
                ELSE 'old'
                END VoteDate,
                COUNT(*) AS Total
                FROM Votes
                WHERE VoteTypeId IN (1, 2, 5)
                GROUP BY PostId, VoteDate
                ) AS VotesDates
                GROUP BY VotesDates.PostId
                HAVING NewVotes > OldVotes
                ) AS VotesByAge ON Posts.Id = VotesByAge.PostId
                WHERE Title NOT IN ('')
                ORDER BY Votes DESC
                LIMIT 10""", conn)

In [6]:
conn.close()

In [7]:
def df_pd_1(Tags):
    df = Tags[['Count', 'TagName']] #wybieramy kolumny
    df = df[df.Count > 1000] #wybieramy wiersze spełniające nasze kryterium
    df = df.sort_values('Count', ascending = False) #ustalamy kolejnosc
    df.index = np.arange(0 ,len(df)) #ustalamy numeracje
    return df

In [8]:
df1 = df_sql_1
df1

,Count,TagName
0,9470,visas
1,5119,usa
2,4601,uk
3,4460,air-travel
4,3503,customs-and-immigration
5,3296,schengen
6,2058,transit
7,1695,passports
8,1665,indian-citizens
9,1517,trains


In [9]:
df1.equals(df_sql_1)

True

In [10]:
def df_pd_2(Users, Posts):
    df_posts = Posts[["OwnerUserId"]] #wybieramy interesujace nas kolumny
    df_users = Users[["Id", "Location"]]
    ans = pd.merge( df_posts, df_users, left_on = "OwnerUserId",right_on = "Id") #laczymy obie ramki razem
    ans= ans[ans.Location.notnull()] #filtrujemy wiersze
    ans = ans.groupby('Location') #grupujemy po kolumnie lokazlizacja
    ans = pd.DataFrame ({'Count': ans.size()}).reset_index() #liczymy liczbe wierszy i tworzymy indeks
    ans = ans.sort_values('Count', ascending = False) #ustalamy kolejnosc odpowiednia
    ans.index = np.arange(0 ,len(ans)) #resetujemy indeksy odpowiednio
    ans = ans.head(10) #wybieramy pierwsze 10 wierszy 
    return ans

In [16]:
df2 = df_pd_2(Users, Posts)
df2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


In [17]:
df2.equals(df_sql_2)

True

In [13]:
def df_pd_3(Badges):
    df = Badges
    df['Year'] = df.Date.str.slice(0,4) #tworzymy nowa kolumne zawierajaca sfromatowana kolumne Date
    df = df[df.Class == 1] #filtrujemy odpoweidnie wiersze
    df = df.groupby(['Name', 'Year']) #grupujemy bo dwoch kolumnach
    df = pd.DataFrame ({'Number': df.size()}).reset_index() #liczymy liczbe rekordow
    df = df.groupby('Year') #ponownie grupujemy
    df = pd.DataFrame ({'TotalNumber': df.Number.sum()}).reset_index() #tworzymy nowo kolumne z sumowanymi wynikami z kol. Number
    df = df.sort_values('TotalNumber') #zmieniamy kolejnosc rekordow
    df.index = np.arange(0 ,len(df)) #ponownie indeksujemy
    return df

In [18]:
df3 = df_pd_3(Badges)
df3

,Year,TotalNumber
0,2011,16
1,2012,23
2,2013,66
3,2021,153
4,2014,197
5,2020,265
6,2015,328
7,2016,509
8,2017,552
9,2018,697


In [19]:
df3.equals(df_sql_3)

True

In [20]:
def df_pd_4(Users, Posts):
    AnsCount = Posts  #tworzymy ramke pomocnicza
    AnsCount = AnsCount[AnsCount.PostTypeId == 2] #filtrujemy wiersze
    AnsCount = AnsCount.groupby('ParentId') #grupujemy
    AnsCount = pd.DataFrame ({'AnswersCount': AnsCount.size()}).reset_index()

    PostAuth = pd.merge( Posts, AnsCount, left_on = "Id",right_on = "ParentId") #tworzymy kolejna ramke łącząc ramkę Posts i AnsCount wzgledem odpowiednich kol.
    PostAuth = PostAuth[['AnswersCount', 'Id', 'OwnerUserId']] #wybieramy kolumny
    PostAuth = PostAuth.groupby(["OwnerUserId"]) #grupujemy 
    PostAuth = pd.DataFrame ({'AverageAnswersCount': PostAuth.AnswersCount.mean()}).reset_index() #tworzymy nową kolumnę

    df = pd.merge(Users, PostAuth, left_on = "AccountId",right_on = "OwnerUserId") #łaczymy dwie ramki
    df = df[['AccountId', 'DisplayName', 'Location', 'AverageAnswersCount']] #wybieramy końcowe kolumny
    df = df.sort_values(['AverageAnswersCount', 'AccountId'], #ustalamy kolejnosc wierszy
              ascending = [False, True])
    df = df.where(pd.notnull(df), None)
    df.index = np.arange(0 ,len(df)) #ustalamy indeksy
    df = df.head(10) #zwracamy 10 indeksów

    return df

In [21]:
df4 = df_pd_4(Users, Posts)
df4

,AccountId,DisplayName,Location,AverageAnswersCount
0,280.0,csmba,"San Francisco, CA",11.0
1,40811.0,vocaro,"San Jose, CA",11.0
2,204.0,Josh,Australia,10.0
3,44093.0,Emma Arbogast,"Salem, OR",10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,19588.0,JD Isaacks,"Atlanta, GA",8.0
6,20473.0,Jeremy Boyd,"Houston, TX",8.0
7,42364.0,Petrogad,None,8.0
8,54571.0,Christian,None,8.0
9,79346.0,Thomas Matthews,California,8.0


In [22]:
df4.equals(df_sql_4)

True

In [23]:
def df_pd_5(Posts, Votes):
    df_votes = Votes
    warunek = df_votes['VoteTypeId'].isin([1, 2, 5])
    df_votes = df_votes[warunek] #filtrujemy wiersze spełniające warunek
    df_votes = df_votes[['PostId', 'CreationDate']] #wybieramy kolumny
    df_votes['VoteDate'] = df_votes.CreationDate.str.slice(0,4) #zmienamy format daty i tworzymy nową kolumnę
    df_votes['VoteDate'].mask(df_votes['VoteDate'] == '2021', 'new', inplace = True) # zmieniamy zawartosc wierszy
    df_votes['VoteDate'].mask(df_votes['VoteDate'] == '2020', 'new', inplace = True)
    df_votes['VoteDate'].mask(df_votes['VoteDate'] != 'new', 'old', inplace = True)
    df_votes = df_votes.groupby(['PostId', 'VoteDate']) #grupujemy dwiema kolumnami
    df_votes = pd.DataFrame ({'Total': df_votes.size()}).reset_index() #tworzymy nową kolumnę
    df_votes['NewVotes'] = df_votes['VoteDate'] #tworzymy kolumnę zawierającą Total jeżeli jest data jest new
    df_votes['NewVotes'].mask(df_votes['NewVotes'] == 'new', df_votes.Total, inplace = True)
    df_votes['NewVotes'].mask(df_votes['NewVotes'] == 'old', 0, inplace = True)
    df_votes['OldVotes'] = df_votes['VoteDate'] #tworzymy kolumnę zawierającą Total jeżeli jest data jest old
    df_votes['OldVotes'].mask(df_votes['OldVotes'] == 'old', df_votes.Total, inplace = True)
    df_votes['OldVotes'].mask(df_votes['OldVotes'] == 'new', 0, inplace = True)
    df_votes = df_votes.groupby('PostId') #grupujemy
    df_votes = pd.DataFrame ({'NewVotes': df_votes.NewVotes.sum(), #tworzymy kolumny z smuowanymi zawartosciami
                         'OldVotes': df_votes.OldVotes.sum(),
                         'Votes': df_votes.Total.sum()}).reset_index()
    df_votes = df_votes[df_votes.NewVotes > df_votes.OldVotes] #filtrujemu wiersze
    VotesByAge = df_votes
    
    df = Posts
    df = df[['Title', 'Id', 'CreationDate']] #wybieramy kolumny
    df['Date'] = df.CreationDate.str.slice(0,10) #tworzymy nowa kolumne ze sformatowana data
    df= df[df.Title.notnull()] #filtrujemy
    df = pd.merge(df, VotesByAge, left_on = "Id",right_on = "PostId") #łączymy dwie rmaki ze sobą
    df = df[['Title', 'Id', 'Date', 'Votes']] #wybieramy cztery kolumny
    df = df.sort_values(['Votes', 'Title'], #ustalamy kolejnosc alfabetyczną dla Title i malejącą dla Votes
          ascending = [False, True])
    df = df.head(10) #wybieramy pierwsze 10 wierszy
    df.index = np.arange(0 ,len(df)) #nowa indeksacja wierszy
    return df

In [24]:
df5 = df_pd_5(Posts, Votes)
df5

,Title,Id,Date,Votes
0,What's the longest distance that can be travel...,151994,2020-01-09,140
1,Considerations for very fragile and expensive ...,157138,2020-05-10,108
2,What are these chair-like things in hotels?,153605,2020-02-10,94
3,What to do if I overstayed my e-visa for Saint...,152418,2020-01-18,84
4,What are the hidden fees of a cruise?,152178,2020-01-13,80
5,What happens when a town is under quarantine a...,152677,2020-01-23,76
6,I've been warned to leave the US within 10 day...,156945,2020-04-28,72
7,"When an individual enters the United States, c...",159298,2020-09-06,68
8,Is Seiryu Miharashi Station the only train sta...,162386,2021-02-04,64
9,Positive drug test in Singapore: How long do I...,163264,2021-04-08,64


In [25]:
df5.equals(df_sql_5)

True

## Podsumowanie

Pakiet Pandas z łatwością sobie radzi z poleceniami SQL'owymi. Oczywiście najszybciej jest po prostu wykonać polecenie korzystając z 'pd.read_sql_query' jednak w analogiczny sposób można odtworzyć polecenia SQL w Pythonie, korzystjąc z pakietu Pandas. Warto zwrócić uwagę, że pomimo innej składni funkcje pisaliśmy podobnie jak w R, sens pozostał taki sam.